In [1]:
import streamlit as st
import pydeck as pdk
import json
import pandas as pd
import random
import geopandas as gpd


## Creating the dummy user interface for the model
Current problems include:
- hexagons not side by side
- map still appearing when running the py file but not in Jupyter

In [2]:
#downloaded from https://martinjc.github.io/UK-GeoJSON/ *ENGLAND > National > \
# Local Authority Districts and converted to GEOJson

with open("England.geojson", "r") as file:
    data = json.load(file)


In [4]:
#Changing the centroid lons and lats as a DataFrame
gdf = gpd.GeoDataFrame.from_features(data["features"])
coordinates = gdf.get_coordinates()
coordinates.columns = ["lng", "lat"]
centroids = gdf.centroid.to_frame()
df = pd.DataFrame()
df["lng"] = gdf.centroid.x
df["lat"] = gdf.centroid.y


In [5]:
#styling
layer = pdk.Layer(
    "HexagonLayer",
    df,
    get_position=["lng", "lat"],
    elevation_scale=0,
    pickable=True,
    elevation_range=[0, 3000],
    filled=True,
    coverage=10,
)


In [6]:
view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=5,
    min_zoom=5,
    max_zoom=15,
    pitch=0,
    bearing=0,
)


In [7]:
# Render without the map
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    map_style="",
    map_provider="",
    api_keys=None,
)


In [8]:
# Streamlit app layout with sliders and titles
st.title("UK, hun?")

# Create a placeholder for the map on the left side
st.sidebar.header("Interactive Hex Map")

# Create sliders for conservative and labor party approval ratings on the right side
st.sidebar.header("Sliders for Polling/Approval Ratings")

conservative_rating = st.sidebar.slider(
    "Conservative Rating", min_value=0, max_value=100, value=50
)
labor_party_rating = st.sidebar.slider(
    "Labor Party Rating", min_value=0, max_value=100, value=50
)

# Display the current values of the sliders
st.sidebar.text(f"Current Conservative Rating: {conservative_rating}%")
st.sidebar.text(f"Current Labor Party Rating: {labor_party_rating}%")

# Display the PyDeck chart using st.pydeck_chart
st.pydeck_chart(r)


2023-11-28 21:40:56.632 
  command:

    streamlit run /home/asia/.pyenv/versions/3.10.6/envs/UK_election/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

## Testing a new layer style

In [9]:
#new layerstyle using h3_hex
h3_layer = pdk.Layer(
    "H3HexagonLayer",
    df,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    get_hexagon="hex",
    get_fill_color="[255 - count, 255, count]",
    get_line_color=[255, 255, 255],
    line_width_min_pixels=2,
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=5,
    min_zoom=5,
    max_zoom=15,
    pitch=0,
    bearing=0,
)

# Render
r_new = pdk.Deck(layers=[h3_layer], initial_view_state=view_state, tooltip={"text": "Count: {count}"})
r_new.to_html("h3_hexagon_layer.html")


In [10]:

# Streamlit app layout
st.title("UK, hun?")

# Create a placeholder for the map on the left side
st.sidebar.header("Interactive Hex Map")

# Create sliders for conservative and labor party approval ratings on the right side
st.sidebar.header("Sliders for Polling/Approval Ratings")
conservative_rating = st.sidebar.slider(
    "Conservative Rating", min_value=0, max_value=100, value=50
)
labor_party_rating = st.sidebar.slider(
    "Labor Party Rating", min_value=0, max_value=100, value=50
)

# Display the current values of the sliders
st.sidebar.text(f"Current Conservative Rating: {conservative_rating}%")
st.sidebar.text(f"Current Labor Party Rating: {labor_party_rating}%")

# Display the PyDeck chart using st.pydeck_chart
st.pydeck_chart(r_new)


DeltaGenerator()

## Trying the Hexbins with Plotly layer styles instead from 
https://plotly.com/python/hexbin-mapbox/

In [11]:
hex_data = "uk-constituencies-2023.json"


In [12]:
hex_df = pd.read_json(hex_data)
hex_df


,layout,version,hexes
E14001063,odd-r,1,"{'n': 'Aldershot', 'r': -40, 'q': 56, 'region'..."
E14001064,odd-r,1,"{'n': 'Aldridge-Brownhills', 'r': -30, 'q': 56..."
E14001065,odd-r,1,"{'n': 'Altrincham and Sale West', 'r': -25, 'q..."
E14001066,odd-r,1,"{'n': 'Amber Valley', 'r': -27, 'q': 58, 'regi..."
E14001067,odd-r,1,"{'n': 'Arundel and South Downs', 'r': -44, 'q'..."
...,...,...,...
W07000108,odd-r,1,"{'n': 'Swansea West', 'r': -37, 'q': 45, 'regi..."
W07000109,odd-r,1,"{'n': 'Torfaen', 'r': -34, 'q': 50, 'region': ..."
W07000110,odd-r,1,"{'n': 'Vale of Glamorgan', 'r': -38, 'q': 47, ..."
W07000111,odd-r,1,"{'n': 'Wrexham', 'r': -30, 'q': 50, 'region': ..."


In [13]:
hex_values = hex_df['hexes'].apply(pd.Series)
hex_values.rename(columns={'n': 'region name'}, inplace=True)


In [14]:
hex_values


,region name,r,q,region,colour
E14001063,Aldershot,-40,56,E12000008,#67E767
E14001064,Aldridge-Brownhills,-30,56,E12000005,#E6007C
E14001065,Altrincham and Sale West,-25,52,E12000002,#1DD3A7
E14001066,Amber Valley,-27,58,E12000004,#00B6FF
E14001067,Arundel and South Downs,-44,61,E12000008,#67E767
...,...,...,...,...,...
W07000108,Swansea West,-37,45,W92000004,#0DBC37
W07000109,Torfaen,-34,50,W92000004,#0DBC37
W07000110,Vale of Glamorgan,-38,47,W92000004,#0DBC37
W07000111,Wrexham,-30,50,W92000004,#0DBC37


In [27]:
import numpy as np
hex_values['randNumCol'] = np.random.randint(0, 3, hex_values.shape[0])


In [15]:
token = 'pk.eyJ1Ijoibm5vYmxlMTMiLCJhIjoiY2xwam91bHNoMDF4dTJpbGIzaTRwMWdubSJ9.t9AstZVQrSa9uFByagdtNw'


In [16]:
import plotly.figure_factory as ff
import plotly.express as px

# px.set_mapbox_access_token(open(token).read())
px.set_mapbox_access_token(token)
df = hex_values


In [37]:
# create_hexbin_mapbox(data_frame, lat, lon, color, nx_hexagon, agg_func, \
# animation_frame, color_discrete_sequence, color_discrete_map, labels, \
# color_continuous_scale, range_color, color_continuous_midpoint, \
# opacity, zoom, center, mapbox_style, title, template, width, \
# height, min_count, show_original_data, original_data_marker)

fig = ff.create_hexbin_mapbox(
    data_frame=df, lat="r", lon="q", #color="randNumCol", #color_discrete_map =["red", "blue", "grey"],
    nx_hexagon=22, opacity=0.5, labels={"color": "color label"},
    min_count=1,
)
fig.show()


In [38]:
import plotly.figure_factory as ff

# Define a custom color scale
custom_color_scale = ["red", "blue", "grey"]

fig = ff.create_hexbin(
    x=df['q'], y=df['r'], C=df['randNumCol'],
    gridsize=(30, 30),  # You can adjust the grid size as needed
    color_scale=custom_color_scale,
    min_count=1,
    opacity=0.5, labels={"color": "color label"},
)

fig.update_layout(title_text='Hexbin Plot without Mapbox')
fig.show()


AttributeError: module 'plotly.figure_factory' has no attribute 'create_hexbin'